In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import pandas
import numpy as np

testfolder = '2018-06-07-16-11-54'


In [ ]:
# velocity plot
df = pandas.read_csv(testfolder+'/_xsens_publisher_node_filter_xs_velocity.csv', sep=';')
vel = df[['time_stamp', 'x', 'y', 'z']].copy()
#vel.plot(x='time_stamp', secondary_y=True)

vel_abs = np.linalg.norm(vel[['x','y']].values, axis=1)
plt.title('Velocity')
plt.xlabel('time')
plt.ylabel('v [m/s]')
plt.plot(df[['time_stamp']], vel_abs)
plt.show()


In [ ]:
import folium
from folium import features

df = pandas.read_csv(testfolder+'/_xsens_publisher_node_filter_xs_latlongalt.csv', sep=';')

points = df[['latitude','longitude']].values.tolist()

#subsample
subsample = 10
points = points[0:-1:subsample]
velocity = vel_abs[0:-1:subsample]

# plot n points (folium has issues for n > 20000)
n = 20000
start = int(0.01*len(points))
points = points[start:start+n]
velocity = velocity[start:start+n]

center = np.mean(points, axis=0).tolist()
my_map = folium.Map(location=center, zoom_start=19, max_zoom=23)

color_line = features.ColorLine(
    positions=points,
    colors=velocity,
    colormap=['lightgreen', 'y', 'orange', 'r', 'purple'],
    weight=1)

color_line.add_to(my_map)


my_map      

In [ ]:
import os
import time
from selenium import webdriver

name = 'traj1'
fn = name+'.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
my_map.save(fn)

browser = webdriver.Firefox()
browser.get(tmpurl)
time.sleep(2)  #Give the map tiles some time to load
browser.save_screenshot(os.path.join('plot', name+'.png'))
browser.quit()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm


r = 6371e3 # m
x = df[['longitude']].values*np.pi/180
y = df[['latitude']].values*np.pi/180

x = r*np.sin(x)*np.cos(np.mean(y))
y = r*np.sin(y)

x -= x[0]
y -= y[0]

subsample = 100
x = x[0:-1:subsample]
y = y[0:-1:subsample]
velocity = vel_abs[0:-1:subsample]

n = 20000
start = int(0.01*len(points))
x = x[start:start+n]
y = y[start:start+n]
velocity = velocity[start:start+n]

# Create a set of line segments so that we can color them individually
# This creates the points as a N x 1 x 2 array so that we can stack points
# together easily to get the segments. The segments array for line collection
# needs to be (numlines) x (points per line) x 2 (for x and y)
points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)
fig, axs = plt.subplots(1, 1, sharex=True, sharey=True)

# Create a continuous norm to map from data points to colors
norm = plt.Normalize(velocity.min(), velocity.max())
lc = LineCollection(segments, cmap='viridis', norm=norm)
# Set the values used for colormapping
lc.set_array(velocity)
lc.set_linewidth(1)
line = axs.add_collection(lc)
axcb = fig.colorbar(line, ax=axs)
axcb.set_label('velocity [m/s]')

axs.set_xlim(x.min(), x.max())
axs.set_ylim(y.min(), y.max())
plt.xlabel('x [m]')
plt.ylabel('y [m]')
plt.savefig('plot/traj1.png')
plt.show()